## Veiew DB in panadas

In [1]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
print(cols)
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
res = results[(results['ticker']!='None')]
res[['id', 'strategy_type', 'options_expiry_date','tab_name', 'er', 'ticker','trigger_price', 'strike_price', 'strike_buy','strike_sell','estimated_premium', 'last_price_when_checked',
         'timestamp_of_price_when_last_checked','scrape_date', 'timestamp_of_trigger','strategy_status', 'price_when_triggered', 'price_when_order_placed','premium_when_last_checked']].tail(12)

['id', 'scrape_date', 'strategy_type', 'tab_name', 'ticker', 'er', 'INTEGER', 'trigger_price', 'strike_price', 'strike_buy', 'strike_sell', 'estimated_premium', 'last_price_when_checked', 'timestamp_of_price_when_last_checked', 'item_id', 'options_expiry_date', 'date_info', 'timestamp_of_trigger', 'strategy_status', 'price_when_triggered', 'price_when_order_placed', 'premium_at_order', 'premium_when_last_checked', 'timestamp_of_order']


/tmp/ipykernel_72871/1121265515.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,id,strategy_type,options_expiry_date,tab_name,er,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,scrape_date,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked
499,839,Bull Put,2025-08-22,Mild Risk 95-97% accuracy > longer expiry,0.0,TQQQ,77.94,sell 74.0 - buy 65.0,65.0,74.0,213.0,88.97,2025-08-06T19:32:39.514511,2025-08-06T11:18:58.930292,None,None,NaN,None,NaN
500,840,Bull Put,2025-08-22,Mild Risk 95-97% accuracy > longer expiry,0.0,TSM,225.48,sell 210.0 - buy 200.0,200.0,210.0,142.0,232.04,2025-08-06T19:32:39.523341,2025-08-06T11:18:59.035383,None,None,NaN,None,NaN
501,841,Bull Put,2025-08-22,Minimal Risk 97-99% accuracy > longer expiry,0.0,AAPL,195.09,sell 185.0 - buy 175.0,175.0,185.0,180.0,213.77,2025-08-06T19:32:39.561404,2025-08-06T11:19:06.119678,None,None,NaN,None,NaN
502,842,Bull Put,2025-08-22,Minimal Risk 97-99% accuracy > longer expiry,0.0,AMZN,210.71,sell 200.0 - buy 190.0,190.0,200.0,145.0,221.36,2025-08-06T19:32:39.571572,2025-08-06T11:19:06.195081,None,None,NaN,None,NaN
503,843,Bull Put,2025-08-22,Minimal Risk 97-99% accuracy > longer expiry,0.0,GOOGL,184.23,sell 175.0 - buy 165.0,165.0,175.0,136.0,196.34,2025-08-06T19:32:39.579383,2025-08-06T11:19:06.272203,None,None,NaN,None,NaN
504,844,Bear Call,2025-08-22,Mild Risk 95-97% accuracy > shorter expiry,0.0,GOOGL,203.95,sell 215.0 - buy 225.0,225.0,215.0,134.0,NaN,None,2025-08-07T07:31:27.187311,None,None,NaN,None,NaN
505,845,Bear Call,2025-08-22,Mild Risk 95-97% accuracy > shorter expiry,0.0,UNH,257.65,sell 280.0 - buy 290.0,290.0,280.0,100.0,NaN,None,2025-08-07T07:31:27.274750,None,None,NaN,None,NaN
506,846,Bear Call,2025-08-29,Mild Risk 95-97% accuracy > longer expiry,0.0,UNH,258.88,sell 275.0 - buy 285.0,285.0,275.0,179.0,NaN,None,2025-08-07T07:31:41.289726,None,None,NaN,None,NaN
507,847,Bull Put,2025-08-22,Mild Risk 95-97% accuracy > shorter expiry,0.0,MSFT,513.61,sell 485.0 - buy 475.0,475.0,485.0,78.0,NaN,None,2025-08-07T07:32:06.724859,None,None,NaN,None,NaN
508,848,Bull Put,2025-08-29,Mild Risk 95-97% accuracy > longer expiry,0.0,TQQQ,81.68,sell 78.0 - buy 70.0,70.0,78.0,159.0,NaN,None,2025-08-07T07:32:20.803537,None,None,NaN,None,NaN


In [2]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
print(cols)
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
res = results[(results['ticker']!='None') & (results['strategy_status']=='order placed')]
res[['id', 'strategy_type', 'tab_name', 'er', 'ticker','trigger_price', 'strike_price', 'strike_buy','strike_sell','estimated_premium', 'options_expiry_date',
         'timestamp_of_price_when_last_checked','scrape_date', 'timestamp_of_trigger','strategy_status', 'price_when_triggered', 'price_when_order_placed','premium_when_last_checked']].tail(9)

['id', 'scrape_date', 'strategy_type', 'tab_name', 'ticker', 'er', 'INTEGER', 'trigger_price', 'strike_price', 'strike_buy', 'strike_sell', 'estimated_premium', 'last_price_when_checked', 'timestamp_of_price_when_last_checked', 'item_id', 'options_expiry_date', 'date_info', 'timestamp_of_trigger', 'strategy_status', 'price_when_triggered', 'price_when_order_placed', 'premium_at_order', 'premium_when_last_checked', 'timestamp_of_order']


,id,strategy_type,tab_name,er,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,options_expiry_date,timestamp_of_price_when_last_checked,scrape_date,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked
424,707,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,0.0,UNH,300.98,sell 280.0 - buy 270.0,270.0,280.0,126.0,2025-07-11,2025-06-19T21:59:52.935890,2025-06-19T07:05:12.464803,2025-06-19T20:31:01.673866,order placed,-1.00,None,126.0
430,713,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,0.0,UNH,303.46,sell 285.0 - buy 275.0,275.0,285.0,95.0,2025-07-03,2025-06-20T21:59:45.680979,2025-06-20T07:21:20.540957,2025-06-20T15:37:36.049769,order placed,303.29,None,95.0
434,717,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,GOOGL,168.81,sell 160.0 - buy 150.0,150.0,160.0,118.0,2025-07-11,2025-06-20T21:59:45.638997,2025-06-20T07:21:28.030847,2025-06-20T15:01:57.743887,order placed,168.79,None,119.5
469,769,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0.0,MSFT,559.14,sell 585.0 - buy 595.0,595.0,585.0,118.0,2025-08-15,2025-08-01T19:59:36.282564,2025-08-01,2025-07-31T05:45:56.872946,order placed,559.33,None,118.0
470,770,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,0.0,UNH,257.19,sell 240.0 - buy 230.0,230.0,240.0,115.0,2025-08-15,2025-08-01T19:59:36.331417,2025-08-01,2025-07-31T13:44:18.361513,order placed,256.89,None,115.0
471,771,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,AAPL,204.28,sell 195.0 - buy 185.0,185.0,195.0,190.0,2025-08-22,2025-08-01T19:59:36.293389,2025-08-01,2025-08-01T14:41:15.866466,order placed,203.91,None,192.5
472,772,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,NKE,74.1,sell 71.0 - buy 61.0,61.0,71.0,102.0,2025-08-22,2025-08-01T19:59:36.298572,2025-08-01,2025-08-01T08:34:37.841907,order placed,72.66,None,102.0
473,773,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0.0,UNH,253.59,sell 240.0 - buy 230.0,230.0,240.0,135.0,2025-08-22,2025-08-01T19:59:36.307437,2025-08-01,2025-07-31T15:10:31.803712,order placed,253.05,None,135.0
480,780,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,0.0,TSM,233.11,sell 225.0 - buy 215.0,215.0,225.0,216.0,2025-08-22,2025-08-01T19:59:36.347981,2025-08-01T12:59:18.349324,2025-08-01T13:49:55.858828,order placed,233.02,None,232.5


## Delete ALL rows in SQLite database

In [3]:
stop

NameError: name 'stop' is not defined

In [ ]:

import sqlite3

conn = sqlite3.connect('option_strategies.db')
c = conn.cursor()

# Delete all rows from table
c.execute('DELETE FROM option_strategies;',)

print('We have deleted', c.rowcount, 'records from the table.')

# Commit the changes to db			
conn.commit()
# Close the connection
conn.close()

We have deleted 20 records from the table.


## Delete table

In [ ]:

# importing sqlite module 
import sqlite3 
  
# create connection to the  
# database geek 
connection = sqlite3.connect('option_strategies.db') 
  
# drop table 
connection.execute("DROP TABLE option_strategies") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully


## Update Row

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (55, 7))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## update estimated premium

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET estimated_premium=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (10, 496))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## update trigger price

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (100, 496))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## update ticker symbol

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET ticker=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, ('JNJ', 192))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET strike_buy=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (583.0, 419))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## Change columns entries 'strategy_status'

In [ ]:
import sqlite3

def update_strategy_status():
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Define the SQL update query
        update_query = """
        UPDATE option_strategies
        SET strategy_status = 'None'
        WHERE strategy_status = 'individual orders placed'
        AND DATE(scrape_date) = '2025-08-01';	
        """
        # 'premium too low';
        
        # Execute the update query
        cursor.execute(update_query)
        conn.commit()
        
        # Report the number of rows updated
        print(f"{cursor.rowcount} rows updated.")
        
    except sqlite3.Error as error:
        print("Error while updating SQLite table:", error)
    finally:
        # Ensure that the connection is closed
        if conn:
            conn.close()

if __name__ == "__main__":
    update_strategy_status()



4 rows updated.


 ## Change columns entries 'scrape_date'

In [ ]:
import sqlite3

def update_scrape_date():
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Update query: change scrape_date to '2025-04-06'
        # where the date portion of scrape_date is '2025-04-03'
        update_query = """
        UPDATE option_strategies
        SET scrape_date = '2025-08-01'
        WHERE DATE(scrape_date) = '2025-07-31';  
        """
        
        # Execute the update query
        cursor.execute(update_query)
        conn.commit()
        
        # Output the number of rows updated
        print(f"{cursor.rowcount} rows updated.")
        
    except sqlite3.Error as error:
        print("Error updating scrape_date:", error)
    finally:
        # Ensure the connection is closed
        if conn:
            conn.close()

if __name__ == "__main__":
    update_scrape_date()



6 rows updated.


## Add New column

In [ ]:
import sqlite3
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def add_premium_column():
    """Add a 'timestamp_of_order ' column to the option_strategies table."""
    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Check if the column already exists
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [col[1] for col in cursor.fetchall()]
        
        if 'timestamp_of_order ' in columns:
            logger.info("Column 'timestamp_of_order  already exists in the table.")
            return
        
        # Add the new column - using REAL type for storing decimal/float values
        cursor.execute("""
            ALTER TABLE option_strategies 
            ADD COLUMN timestamp_of_order DATETIME
        """)
        
        conn.commit()
        logger.info("Successfully added 'timestamp_of_order' column to option_strategies table.")
        
    except sqlite3.Error as e:
        logger.error(f"SQLite error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    add_premium_column()

2025-04-06 08:14:48,843 - ERROR - SQLite error: no such table: option_strategies


## Query a section of the database

In [ ]:


start_date = '2025-03-29'
dat = sqlite3.connect('option_strategies.db')
query_text = f"""
    SELECT * FROM option_strategies 
    WHERE date(scrape_date) = '{start_date}'
    AND strategy_status = 'triggered'
    """
query = dat.execute(query_text)
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

OperationalError: no such table: option_strategies

## Delete all row except 1

In [ ]:
import sqlite3

# Connect to your database
conn = sqlite3.connect('option_strategies.db')
cur= conn.cursor()

# Delete all rows except the one where id = 312
cur.execute("DELETE FROM option_strategies WHERE id != ?", (312,))

# Commit changes and close connection
conn.commit()
conn.close()

print("Rows deleted except the one with id = 312.")

Rows deleted except the one with id = 312.


## Delete rows from scrape date

In [ ]:
import sqlite3

date_to_delete = '2025-08-05'

# Connect to your database
conn = sqlite3.connect('option_strategies.db')
cur = conn.cursor()

# Delete rows for the specific date
cur.execute("DELETE FROM option_strategies WHERE DATE(scrape_date) = ?", (date_to_delete,))

# Get number of rows deleted
rows_deleted = cur.rowcount

# Commit changes and close connection
conn.commit()
conn.close()

print(f"{rows_deleted} rows deleted.")



0 rows deleted.


## Change Ticker symbol

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET ticker=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, ('WMT', 179))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

In [ ]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET ticker=?, trigger_price=?, strike_buy=?, strike_sell=? WHERE id = ?'
try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, ('BBY', 650, 680, 690, 312))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)